# The Relationship between Turnout and Topics

In [1]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      StataNow 19.5
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2025 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Unlimited-user 2-core network, expiring  9 Sep 2026
Serial number: 501909305069
  Licensed to: 罗奕辰
               UCL

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [2]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [3]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local dep non_whale_participation whale_participation

* Independent variables
local topic protocol_security treasury_expenditure user_incentive_increase tokenomics voting_mechanism_change

* Label variables
label var user_incentive_increase "\${\it User Incentive Increase}_{i,t}\$"
label var treasury_expenditure "\${\it Treasury Expenditure}_{i,t}\$"
label var voting_mechanism_change "\${\it Voting Mechanism Change}_{i,t}\$"
label var protocol_security "\${\it Protocol Security}_{i,t}\$"
label var tokenomics "\${\it Tokenomics}_{i,t}\$"


. 
. import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear
(encoding automatically selected: ISO-8859-1)
(51 vars, 2,841 obs)

. 
. * Parse date
. gen day = date(date, "YMD")

. format day %td

. gen month = month(day)

. gen quarter = quarter(day)

. 
. gen year = year(day)

. encode gecko_id, gen(token)

. encode space,    gen(dao)

. 
. * Dependent variables
. local dep non_whale_participation whale_participation

. 
. * Independent variables
. local topic protocol_security treasury_expenditure user_incentive_increase to
> kenomics voting_mechanism_change

. 
. * Label variables
. label var user_incentive_increase "\${\it User Incentive Increase}_{i,t}\$"

. label var treasury_expenditure "\${\it Treasury Expenditure}_{i,t}\$"

. label var voting_mechanism_change "\${\it Voting Mechanism Change}_{i,t}\$"

. label var protocol_security "\${\it Protocol Security}_{i,t}\$"

. label var tokenomics "\${\it Tokenomics}_{i,t}\$"

. 


## Proposal Characteristics

In [4]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

* VIF test
qui reg non_whale_participation `topic'
estat vif

foreach d of local dep {
    * Run regression
    reghdfe `d' `topic', absorb(year dao) vce(cluster token)
    eststo `d'

    * Tag FE rows (Y/N) for this spec
    estadd local fe_token "Y"
    estadd local fe_time  "Y"
}

* Export LaTeX table
esttab                                                      ///
    non_whale_participation whale_participation                         ///
    using "$TABLES/reg_participation_topic.tex", replace          ///
    se star(* 0.10 ** 0.05 *** 0.01) nogaps nocon           ///
    b(%9.3f) se(%9.2f)                                      ///
    label nogaps booktabs nonotes noomitted eqlabels(none)  ///
    mtitles("\${\it Participation}^{Small}_{i,t}\$"               ///
    "\${\it Participation}^{Whale}_{i,t}\$")                      ///  
    substitute("\_" "_")                                    ///
    stats(fe_token fe_time N r2_a,                          ///
        fmt(0 0 %9.0fc %9.3f)                                   ///
         labels("Token FE" "Year FE" "Observations" "Adjusted R²"))


. 
. ******************************************************
. * LOGISTIC REGRESSIONS
. ******************************************************
. 
. eststo clear

. 
. * VIF test
. qui reg non_whale_participation `topic'

. estat vif

    Variable |       VIF       1/VIF  
-------------+----------------------
  tokenomics |      1.23    0.811829
user_incen~e |      1.20    0.830961
treasury_e~e |      1.09    0.917481
voting_mec~e |      1.04    0.966111
protocol_s~y |      1.02    0.984416
-------------+----------------------
    Mean VIF |      1.12

. 
. foreach d of local dep {
  2.     * Run regression
.     reghdfe `d' `topic', absorb(year dao) vce(cluster token)
  3.     eststo `d'
  4. 
.     * Tag FE rows (Y/N) for this spec
.     estadd local fe_token "Y"
  5.     estadd local fe_time  "Y"
  6. }
(dropped 12 singleton observations)
(MWFE estimator converged in 7 iterations)

HDFE Linear regression                            Number of obs   =      2,667
Absorbing 2 HDFE groups 